In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
# cost or loss function
def cost(Y, Yhat):
    return np.mean((Yhat - Y) ** 2)

In [3]:
path = '../../data/KNN_Linear_Regression/vidu4_lin_reg.txt'

df = pd.read_csv(path, sep=" ")
df.head()

,ID,TUOI,BMI,HA,GLUCOSE,CHOLESTEROL,BEDAYNTM
0,1,56,21,160,14.0,6.00,1.95
1,2,76,18,150,12.0,4.97,1.33
2,3,63,16,160,4.4,6.39,0.83
3,4,78,20,100,4.0,7.00,2.00
4,5,87,20,110,4.6,4.10,1.30


In [4]:
X = df.drop(['ID', 'BEDAYNTM'], axis=1)
y = df['BEDAYNTM']

X_train = X[:80]
y_train = y[:80]

X_test = X[80:]
y_test = y[80:]

In [5]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_test = y_test.to_numpy().reshape(-1, 1)

**Huấn luyện mô hình `ANN` với dữ liệu tập Train**

In [6]:
d0 = 5
d1 = h = 100 # size of hidden layer
d2 = C = 1
# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))

N = X_train.T.shape[1]
eta = 0.001 # learning rate

for i in range(10000):
    ## Feedforward
    Z1 = np.dot(W1.T, X_train.T) + b1
    A1 = np.where(Z1 > 0, Z1, 0.01 * Z1)  # LeakyReLU
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = Z2

    # print loss after each 1000 iterations
    if i %1000 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(y_train, Yhat)
        print("iter %d, loss: %f" %(i, loss))

    # backpropagation
    E2 = (Yhat - y_train.T)/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1 = np.where(Z1 > 0, E1, 0.01 * E1)  # Gradient of LeakyReLU
    dW1 = np.dot(X_train.T, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)
    
    # Gradient clipping (To avoid booming gradient)
    clip_value = 1.0
    dW1 = np.clip(dW1, -clip_value, clip_value)
    dW2 = np.clip(dW2, -clip_value, clip_value)
    db1 = np.clip(db1, -clip_value, clip_value)
    db2 = np.clip(db2, -clip_value, clip_value)
    
    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2

iter 0, loss: 1.010095
iter 1000, loss: 0.152525
iter 2000, loss: 0.152228
iter 3000, loss: 0.152043
iter 4000, loss: 0.151956
iter 5000, loss: 0.151910
iter 6000, loss: 0.151891
iter 7000, loss: 0.151872
iter 8000, loss: 0.151845
iter 9000, loss: 0.151823


**Dự đoán của mô hình với dữ liệu trong tập Train**

In [7]:
Z1 = np.dot(W1.T, X_train.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2

Z2

array([[1.12082839, 1.24601952, 1.0868554 , 1.0878102 , 1.19604745,
        1.17547033, 1.02277586, 1.05398375, 1.17159478, 1.22952681,
        0.67345556, 1.14943356, 1.2203613 , 1.0306111 , 0.98057657,
        0.74384184, 1.06850931, 1.22792687, 1.06068595, 1.06975716,
        1.25396264, 1.17698511, 1.36018921, 0.80959447, 1.34543186,
        1.21268292, 1.00506598, 0.95631103, 1.26018994, 0.96352228,
        1.02598717, 1.13040661, 1.14361081, 0.99944447, 0.8680264 ,
        1.20500185, 1.04187736, 1.0666443 , 1.16356392, 1.10377833,
        1.33487152, 1.03606611, 0.76597022, 0.98452737, 1.0173377 ,
        1.21657976, 0.92779374, 0.98527958, 1.02356837, 0.75357098,
        1.01366941, 1.12439719, 1.02811936, 1.10651459, 1.21909649,
        1.12866882, 0.79185092, 1.04414977, 0.74939202, 1.0745346 ,
        0.99203311, 1.0693366 , 0.97805633, 1.09374101, 0.94531887,
        0.89337238, 1.00360539, 1.0908511 , 0.88185608, 1.0640148 ,
        1.17395933, 1.460774  , 1.13279713, 0.93

**Đánh giá độ chính xác**

* R square = 0.1908 cho thấy các biến độc lập giải thích được khoảng 19% sự biến thiên của biến phụ thuộc.

* MSE = 0.1043 là khá cao khi xét trong khoảng giá trị dự đoán.

In [8]:
print("R square:", r2_score(y_train, Z2[0]))
print("MSE:", mean_squared_error(y_train, Z2[0]))

R square: 0.19083418968762578
MSE: 0.10436912679693662


**Dự đoán của mô hình với dữ liệu trong tập Test**

In [9]:
Z1 = np.dot(W1.T, X_test.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2

Z2

array([[0.90563399, 0.79352247, 1.21404992, 0.95355705, 0.95810554,
        1.05056676, 1.20995994, 1.050404  , 0.99740187, 0.68808774,
        0.79975729, 0.76034558, 1.18836302, 1.06836671, 0.98866149,
        1.03041837, 1.15396368, 1.19221038, 1.3060959 , 1.28071263]])

**Đánh giá độ chính xác**

* R square = 0.2154 cho thấy các biến độc lập giải thích được khoảng 21% sự biến thiên của biến phụ thuộc.

* MSE = 0.2271 tăng lên rất nhiều so với MSE khi đánh giá với tập Train.

=> Cho thấy mô hình đang bị overfitting với dữ liệu tập Train.

In [10]:
print("R square:", r2_score(y_test, Z2[0]))
print("MSE:", mean_squared_error(y_test, Z2[0]))

R square: 0.21549617489199424
MSE: 0.2271238597925378


**Sử dụng hồi quy tuyến tính**

In [11]:
linR = LinearRegression()

linR.fit(X_train, y_train)

y_pred = linR.predict(X_test)

print("R square:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

R square: 0.21453893938640334
MSE: 0.22740099167615907


**So sánh độ chính xác của `Linear Regression` và `ANN`**

* R square và MSE của mô hình hồi quy tuyến tính tương đương với R square của mô hình ANN khi dự đoán với tập Test.
